In [8]:
# ----------------------------
# 1. Helper: clear output
# ----------------------------
import os, sys

def clear_output():
    """
    Clear output for both Jupyter and plain console.
    """
    os.system('cls' if os.name == 'nt' else 'clear')
    if 'ipykernel' in sys.modules:
        try:
            from IPython.display import clear_output as _clear
            _clear(wait=True)
        except Exception:
            pass

# ----------------------------
# 2. Paths and EnergyPlus setup
# ----------------------------

# >>> CHANGE THIS to your notebooks folder if needed <<<
root_dir = r"C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks"
os.chdir(root_dir)  # make sure we are in the project root

print("Project root:", root_dir)
print("Files in project root:", os.listdir(root_dir))

# EnergyPlus installation path (adjust if different on your machine)
eplus_root = r"C:\EnergyPlusV25-1-0"

iddfile = os.path.join(eplus_root, "Energy+.idd")
from eppy.modeleditor import IDF
IDF.setiddname(iddfile)

base_idf_path = os.path.join(root_dir, "model_week4.idf")
assert os.path.exists(base_idf_path), f"Base IDF not found: {base_idf_path}"
print("Using base IDF:", base_idf_path)

# ----------------------------
# 3. ESO helper: extract cooling kWh
# ----------------------------

import esoreader
import pandas as pd
from itertools import product
from tqdm import trange

class ESO:
    def __init__(self, path):
        self.dd, self.data = esoreader.read(path)

    def read_var(self, variable, frequency="Hourly"):
        return [
            {
                "key": k,
                "series": self.data[self.dd.index[frequency, k, variable]],
            }
            for _f, k, _v in self.dd.find_variable(variable)
        ]

    def total_kwh(self, variable, frequency="Hourly"):
        j_per_kwh = 3_600_000
        results = self.read_var(variable, frequency)
        return sum(sum(s["series"]) for s in results) / j_per_kwh

# ----------------------------
# 4. Quick sanity check of base IDF
# ----------------------------

idf_test = IDF(base_idf_path)
print("Zones:", [z.Name for z in idf_test.idfobjects["ZONE"]])
print("Number of MATERIAL objects:", len(idf_test.idfobjects["MATERIAL"]))
print("Number of WINDOW objects:", len(idf_test.idfobjects["WINDOW"]))
print("Number of WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM:",
      len(idf_test.idfobjects["WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM"]))
print("Number of PEOPLE objects:", len(idf_test.idfobjects["PEOPLE"]))
print("Number of LIGHTS objects:", len(idf_test.idfobjects["LIGHTS"]))
print("Number of SCHEDULE:COMPACT objects:",
      len(idf_test.idfobjects["SCHEDULE:COMPACT"]))

# ----------------------------
# 5. Parameter grid
# ----------------------------

Location = ["San Francisco", "Sacramento", "Chicago", "New York"]
Wall_Rvalue = [14.3, 21.9, 29.7]          # ft²·°F·hr/Btu
WWR = [0.25, 0.40, 0.60]
SHGC = [0.25, 0.40, 0.60]
Occupancy_Density = [0.04, 0.06, 0.08]    # people/m²
Lights_Density = [7.5, 9.0]               # W/m²
Cooling_Setpoint = [24, 26, 28]           # °C

keys = [
    "Location",
    "Wall_Rvalue",
    "WWR",
    "SHGC",
    "Occupancy_Density",
    "Lights_Density",
    "Cooling_Setpoint",
]

combos = [
    dict(zip(keys, vals))
    for vals in product(
        Location,
        Wall_Rvalue,
        WWR,
        SHGC,
        Occupancy_Density,
        Lights_Density,
        Cooling_Setpoint,
    )
]

params_df = pd.DataFrame(combos)

# Convert R-value from imperial to SI (m²·K/W)
params_df["Wall_Rvalue_m2K_per_W"] = params_df["Wall_Rvalue"] * 0.1761108

print("Total number of cases:", len(params_df))
display(params_df.head())

# ----------------------------
# 6. Weather file mapping
# ----------------------------

MAP_WEATHER = {
    "San Francisco": os.path.join(root_dir, r"weather_data\USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"),
    "Sacramento":    os.path.join(root_dir, r"weather_data\USA_CA_Sacramento.Exec.AP.724830_TMY3.epw"),
    "Chicago":       os.path.join(root_dir, r"weather_data\USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw"),
    "New York":      os.path.join(root_dir, r"weather_data\USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw"),
}

print("\nChecking weather files:")
for loc, p in MAP_WEATHER.items():
    print(f"  {loc}: {p}  ->  exists? {os.path.exists(p)}")

# ----------------------------
# 7. Mapping WWR -> window geometry
# ----------------------------

MAP_WWR = {
    0.25: [2.0, 3.75],  # [starting_x, length]
    0.40: [2.0, 6.0],
    0.60: [0.5, 9.0],
}

# ----------------------------
# 8. Function to modify IDF
# ----------------------------

def modify_idf(idf, modify_dict):
    """
    Apply this case's parameters to the IDF.
    Assumes a structure similar to the class example.
    """
    # 1) Wall R-value -> change conductivity of external wall material
    materials = idf.idfobjects["MATERIAL"]
    wall_mat = materials[-1]  # assume last material is external "Simple Wall"
    R_m2K_per_W = modify_dict["Wall_Rvalue_m2K_per_W"]
    thickness = wall_mat["Thickness"]  # m
    new_k = thickness / R_m2K_per_W    # W/m·K
    wall_mat["Conductivity"] = new_k

    # 2) Window-to-wall ratio (WWR) -> adjust south-facing window width & position
    windows = idf.idfobjects["WINDOW"]
    new_x, new_length = MAP_WWR[modify_dict["WWR"]]
    windows[0]["Starting_X_Coordinate"] = new_x
    windows[0]["Length"] = new_length

    # 3) SHGC -> simple glazing system
    glazings = idf.idfobjects["WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM"]
    glazings[0]["Solar_Heat_Gain_Coefficient"] = modify_dict["SHGC"]

    # 4) Occupancy density (people/m²)
    people = idf.idfobjects["PEOPLE"]
    people[0]["Number_of_People_Calculation_Method"] = "People/Area"
    people[0]["People_per_Floor_Area"] = modify_dict["Occupancy_Density"]

    # 5) Lighting power density (W/m²)
    lights = idf.idfobjects["LIGHTS"]
    lights[0]["Watts_per_Floor_Area"] = modify_dict["Lights_Density"]

    # 6) Cooling setpoint (disable heating, set cooling)
    schedules = idf.idfobjects["SCHEDULE:COMPACT"]
    # This assumes first schedule is heating, second is cooling.
    schedules[0]["Field_6"] = 12.0                      # almost disable heating
    schedules[1]["Field_6"] = modify_dict["Cooling_Setpoint"]

    # 7) Weather file
    loc = modify_dict["Location"]
    idf.epw = MAP_WEATHER[loc]

    return idf

# ----------------------------
# 9. Temp simulation folder
# ----------------------------

sim_base_dir = os.path.join(root_dir, "final_project_ep_results")
os.makedirs(sim_base_dir, exist_ok=True)
print("\nSimulation temp folder:", sim_base_dir)

# ----------------------------
# 10. Main simulation loop
# ----------------------------

results_df = pd.DataFrame(columns=["Total_Cooling_kwh"])

for i in trange(len(params_df)):
    modify_dict = params_df.iloc[i].to_dict()

    # fresh copy of base IDF each time
    idf = IDF(base_idf_path)
    idf = modify_idf(idf, modify_dict)

    sim_folder = os.path.join(sim_base_dir, f"{i}")
    os.makedirs(sim_folder, exist_ok=True)

    # save a *stable* IDF into the simulation folder
    sim_idf_path = os.path.join(sim_folder, "in.idf")
    idf.saveas(sim_idf_path)

    # run EnergyPlus from that IDF, results go into sim_folder
    idf.idfname = sim_idf_path  # tell eppy this is the file to use
    idf.run(
        expandobjects=True,
        output_directory=sim_folder,
    )

    # now read the ESO file from sim_folder
    eso_path = os.path.join(sim_folder, "eplusout.eso")
    if not os.path.exists(eso_path):
        raise FileNotFoundError(f"ESO file not found for case {i}: {eso_path}")

    eso = ESO(eso_path)
    cooling_kwh = eso.total_kwh("DistrictCooling:Facility", "TimeStep")
    results_df.loc[i, "Total_Cooling_kwh"] = cooling_kwh

# ----------------------------
# 11. Save outputs
# ----------------------------

all_parameters = params_df[keys].copy()

results_path = os.path.join(root_dir, "results.csv")
params_path = os.path.join(root_dir, "all_parameters.csv")

results_df.to_csv(results_path, index_label="case_id")
all_parameters.to_csv(params_path, index_label="case_id")

print("\nSaved output files:")
print("  ", results_path)
print("  ", params_path)


Project root: C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks
Files in project root: ['.ipynb_checkpoints', 'final_project_ep_results', 'final_project_generate_models.ipynb', 'model_week4.idf', 'NB1 Generate Models.ipynb', 'NB2 ML.ipynb', 'test_run', 'weather_data']
Using base IDF: C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\model_week4.idf
Zones: ['zone 1']
Number of MATERIAL objects: 6
Number of WINDOW objects: 1
Number of WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM: 1
Number of PEOPLE objects: 1
Number of LIGHTS objects: 1
Number of SCHEDULE:COMPACT objects: 7
Total number of cases: 1944


,Location,Wall_Rvalue,WWR,SHGC,Occupancy_Density,Lights_Density,Cooling_Setpoint,Wall_Rvalue_m2K_per_W
0,San Francisco,14.3,0.25,0.25,0.04,7.5,24,2.518384
1,San Francisco,14.3,0.25,0.25,0.04,7.5,26,2.518384
2,San Francisco,14.3,0.25,0.25,0.04,7.5,28,2.518384
3,San Francisco,14.3,0.25,0.25,0.04,9.0,24,2.518384
4,San Francisco,14.3,0.25,0.25,0.04,9.0,26,2.518384



Checking weather files:
  San Francisco: C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\weather_data\USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw  ->  exists? True
  Sacramento: C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\weather_data\USA_CA_Sacramento.Exec.AP.724830_TMY3.epw  ->  exists? True
  Chicago: C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\weather_data\USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw  ->  exists? True
  New York: C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\weather_data\USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw  ->  exists? True

Simulation temp folder: C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\final_project_ep_results


  0%|                                                                                         | 0/1944 [00:00<?, ?it/s]


C:\EnergyPlusV25-1-0\energyplus.exe --weather C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\weather_data\USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw --output-directory C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\final_project_ep_results\0 --idd C:\EnergyPlusV25-1-0\Energy+.idd --expandobjects C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\final_project_ep_results\0\in_b4dc61.idf



...


C:\EnergyPlusV25-1-0\energyplus.exe --weather C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\weather_data\USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw --output-directory C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\final_project_ep_results\1942 --idd C:\EnergyPlusV25-1-0\Energy+.idd --expandobjects C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\final_project_ep_results\1942\in_d3d3cd.idf



100%|███████████████████████████████████████████████████████████████████████████▉| 1943/1944 [2:43:04<00:03,  3.76s/it]


C:\EnergyPlusV25-1-0\energyplus.exe --weather C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\weather_data\USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw --output-directory C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\final_project_ep_results\1943 --idd C:\EnergyPlusV25-1-0\Energy+.idd --expandobjects C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\final_project_ep_results\1943\in_188691.idf



100%|████████████████████████████████████████████████████████████████████████████| 1944/1944 [2:43:08<00:00,  5.04s/it]


Saved output files:
   C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\results.csv
   C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\all_parameters.csv


In [2]:
!pip install eppy


In [7]:
import os

eso_path = r"C:\Users\jerry\Desktop\projects\Arch 298 Energy Plus\Final_Project\notebooks\final_project_ep_results\0\eplusout.eso"
print("ESO exists?", os.path.exists(eso_path))


ESO exists? True
